## Causal Tracing

Preliminary experiemnts to determine type system knowledge in Code LLMs

In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=0
!echo $CUDA_VISIBLE_DEVICES
import torch
torch.cuda.is_available()

env: CUDA_VISIBLE_DEVICES=0
0


True

In [6]:
import os, re, json
import torch, numpy
from collections import defaultdict

import sys
sys.path.append("~")
sys.path.append("..")
sys.path.append("../myrome")

from model_utils import *
# CACHE_DIR = "/scratch/lucchetti.f/models/"
# os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR

import sys

DATA_DIR = 'data'
from myrome.experiments.causal_trace import (
    ModelAndTokenizer,
    layername,
    guess_subject,
    plot_trace_heatmap
)
from myrome.experiments.causal_trace import (
    make_inputs,
    decode_tokens,
    find_token_range,
    predict_token,
    predict_from_input,
    collect_embedding_std,
    trace_with_patch,
    trace_important_states,
    calculate_hidden_flow
)
from myrome.dsets import KnownsDataset
from myrome.util.generate import generate_fast, generate_interactive
from model_utils import print_by_line

## check has GPTBigCodeConfig
from transformers import GPTBigCodeConfig

torch.set_grad_enabled(False)

In [7]:
from trace_model import ModelLoader
from pathlib import Path

model_name= "bigcode/starcoder"
# model_name= "bigcode/santacoder"
# model_path = "../../microstarcoder"

tm = ModelLoader(model_name, is_remote=False, quiet=False)

model=tm.model
tokenizer=tm.tokenizer

Loading checkpoint shards: 100%|██████████| 7/7 [00:26<00:00,  3.75s/it]


transformer.wte.weight torch.Size([49152, 6144]) cuda:0
transformer.wpe.weight torch.Size([8192, 6144]) cuda:0
transformer.h.0.ln_1.weight torch.Size([6144]) cuda:0
transformer.h.0.ln_1.bias torch.Size([6144]) cuda:0
transformer.h.0.attn.c_attn.weight torch.Size([6400, 6144]) cuda:0
transformer.h.0.attn.c_attn.bias torch.Size([6400]) cuda:0
transformer.h.0.attn.c_proj.weight torch.Size([6144, 6144]) cuda:0
transformer.h.0.attn.c_proj.bias torch.Size([6144]) cuda:0
transformer.h.0.ln_2.weight torch.Size([6144]) cuda:0
transformer.h.0.ln_2.bias torch.Size([6144]) cuda:0
transformer.h.0.mlp.c_fc.weight torch.Size([24576, 6144]) cuda:0
transformer.h.0.mlp.c_fc.bias torch.Size([24576]) cuda:0
transformer.h.0.mlp.c_proj.weight torch.Size([6144, 24576]) cuda:0
transformer.h.0.mlp.c_proj.bias torch.Size([6144]) cuda:0
transformer.h.1.ln_1.weight torch.Size([6144]) cuda:0
transformer.h.1.ln_1.bias torch.Size([6144]) cuda:0
transformer.h.1.attn.c_attn.weight torch.Size([6400, 6144]) cuda:0
trans

In [8]:
check_devs()

62136934400 / 84979089408 used for device 0, reserved 62138613760


## Examining traces

### Ideas
- grab layers that give correct output in trace
- measure k value (Rank) of correct output

In [9]:
prompts = ["int x ="]

n = 4 # num transformer blocks
txt, ret_dict = tm.sample_generate(
    prompts,
    max_out_len=6,
    argmax_greedy= True,
    debug = True,
    
    request_activations= [tm.layer_name_format.format(i) for i in range(2,3)],
)
txt, ret_dict

/home/franlucc/codetrace/notebooks/../trace_model.py:150: UserWarning: The model `bigcode/starcoder` of type `gpt_bigcode` already implements or can't utilize `use_cache` for fast generation. Setting `use_cache = False`.
  warnings.warn(f"The model `{self.model_name}` of type `{self.model_type}` already implements or can't utilize `use_cache` for fast generation. Setting `use_cache = False`.")


prompt <0> ==> [(' ', 0.6333), (' -', 0.0318), (' (', 0.0177), (' x', 0.0172), (' %', 0.0146)]
prompt <0> ==> [('0', 0.7563), ('1', 0.1451), ('2', 0.0336), ('3', 0.0192), ('5', 0.0137)]
prompt <0> ==> [(';', 0.9095), (',', 0.0309), (';\\', 0.0153), (' ;', 0.0116), ('\n', 0.0028)]


(['int x = 0;'],
 {'input_tokenized': [[('int', 410), (' x', 816), (' =', 280)]],
  'generated_tokens': [[[{'token': ' ', 'id': 225, 'p': 0.6333003044128418},
     {'token': ' -', 'id': 429, 'p': 0.03179261460900307},
     {'token': ' (', 'id': 308, 'p': 0.017688540741801262},
     {'token': ' x', 'id': 816, 'p': 0.017215397208929062},
     {'token': ' %', 'id': 904, 'p': 0.014598190784454346}],
    [{'token': '0', 'id': 34, 'p': 0.7562679052352905},
     {'token': '1', 'id': 35, 'p': 0.1451088786125183},
     {'token': '2', 'id': 36, 'p': 0.033597081899642944},
     {'token': '3', 'id': 37, 'p': 0.019215252250432968},
     {'token': '5', 'id': 39, 'p': 0.013744757510721684}],
    [{'token': ';', 'id': 45, 'p': 0.9095149040222168},
     {'token': ',', 'id': 30, 'p': 0.03092985786497593},
     {'token': ';\\', 'id': 8871, 'p': 0.015337608754634857},
     {'token': ' ;', 'id': 2082, 'p': 0.011616121046245098},
     {'token': '\n', 'id': 203, 'p': 0.002759486436843872}]]],
  'activations'

## Logit lens

In [10]:
print(model)
prompt = "static int func(){\nint x = 0;\nint y = 1;\nint z ="
text = generate_interactive(model, tokenizer, use_logit_lens=True, top_k=50)

GPTBigCodeForCausalLM(
  (transformer): GPTBigCodeModel(
    (wte): Embedding(49152, 6144)
    (wpe): Embedding(8192, 6144)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-39): 40 x GPTBigCodeBlock(
        (ln_1): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
        (attn): GPTBigCodeAttention(
          (c_attn): Linear(in_features=6144, out_features=6400, bias=True)
          (c_proj): Linear(in_features=6144, out_features=6144, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTBigCodeMLP(
          (c_fc): Linear(in_features=6144, out_features=24576, bias=True)
          (c_proj): Linear(in_features=24576, out_features=6144, bias=True)
          (act): GELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((6144,), eps=1

In [7]:
check_devs()

4611912704 / 34079899648 used for device 0, reserved 4632608768
